In [ ]:
import openai
from dotenv import load_dotenv
import gradio as gr
import os
import json
from datetime import datetime

import httpx
from fastapi import FastAPI
import uvicorn
import threading

load_dotenv('.env',override=True)

In [ ]:
app = FastAPI()

@app.post('/mock/ticket')
def booking(payload:dict):
  dt = datetime.strptime(payload.get('date'), "%Y/%m/%d") 
  isoStr = dt.date().isoformat()
  return {"status": "success", "order_id": f"MOCK-FLIGHT-{isoStr}-001"}

# start server
def run():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=run, daemon=True)
thread.start()

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = openai.OpenAI(api_key=openai_api_key)

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

async def booking_flight(departure, destination, date):
  print(f"Book the Flight Automatically, {departure} to {destination} at {date}.")
  reqBody = {
    "departure": departure,
    "destination": destination,
    "date": date
  }
  async with httpx.AsyncClient() as client:
    res = await client.post('http://127.0.0.1:8000/mock/ticket', json=reqBody)
    print(res.text)
    return res.text
  
book_function = {
    "name": "booking_flight",
    "description": "async function for booking the flight ticket for customers and it will return the status and id of flight. Call this function whenever you were asked to book the flight, and you will automatically tell the status of the order and the book number! if customers don't provide their departure or destination or date you should inquire them courteous. Note that the date format you should keep them with %Y/%m/%d. for example when a customer asks 'Please help me book the ticket from <departure> to <destination>'",
    "parameters": {
        "type": "object",
        "properties": {
            "departure": {
                "type": "string",
                "description": "The city where the customer departure",
            },
            "destination": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "date": {
                "type": "string",
                "description": "The date of the flight ",
            },
        },
        "required": ["destination", "departure", "date"],
        "additionalProperties": False
    }
}

async def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    departure = arguments.get('departure')
    destination = arguments.get('destination')
    date = arguments.get('date')
    res = await booking_flight(departure, destination, date)
    response = {
        "role": "tool",
        "content": json.dumps(res),
        "tool_call_id": tool_call.id
    }
    return response

tools = [{"type": "function", "function": book_function}]

In [ ]:
res = await booking_flight('Taiwan', "NewYork", "2025/12/03")

In [ ]:
async def chat(message, history):
  messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
  res = llm.chat.completions.create(messages=messages,
                                    model="gpt-4.1-mini",
                                    tools=tools)
  
  if res.choices[0].finish_reason == 'tool_calls':
    message = res.choices[0].message
    toolResponse = await handle_tool_call(message)
    messages.append(message)
    messages.append(toolResponse)
    res = llm.chat.completions.create(messages=messages,
                                    model="gpt-4.1-mini")

  return res.choices[0].message.content

gr.ChatInterface(fn=chat,type="messages").launch()